In [1]:
import csv
import os
import numpy as np
import tensorflow as tf
from random import randint
from tensorflow import keras
from keras.utils import np_utils, plot_model, to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, concatenate

In [2]:
# returns train, inference_encoder and inference_decoder models
def define_models(dim_p, n_units):
	# define training encoder
	enc_in_o = Input(shape=(None, 1))
	enc_in_q = Input(shape=(None, 1))
	enc_in_p = Input(shape=(None, dim_p))
	encoder_inputs = concatenate([enc_in_o, enc_in_q, enc_in_p])
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	
	# define training decoder
	dec_in_o = Input(shape=(None, 1))
	dec_in_q = Input(shape=(None, 1))
	dec_in_p = Input(shape=(None, dim_p))
	decoder_inputs = concatenate([dec_in_o, dec_in_q, dec_in_p])
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	out_o = TimeDistributed(Dense(1, activation='sigmoid'), name='tr_out_o')(decoder_outputs)#act relu
	out_q = TimeDistributed(Dense(1, activation='sigmoid'), name='tr_out_q')(decoder_outputs)
	out_p = TimeDistributed(Dense(dim_p, activation='softmax'), name='tr_out_p')(decoder_outputs)
	
	model = Model([enc_in_o, enc_in_q, enc_in_p, dec_in_o, dec_in_q, dec_in_p], [out_o, out_q, out_p])
	
	# define inference encoder
	encoder_model = Model([enc_in_o, enc_in_q, enc_in_p], encoder_states)
	
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	out_o = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)#act relu
	out_q = TimeDistributed(Dense(1, activation='sigmoid'))(decoder_outputs)
	out_p = TimeDistributed(Dense(dim_p, activation='softmax'))(decoder_outputs)
	
	decoder_model = Model([dec_in_o, dec_in_q, dec_in_p] + decoder_states_inputs, [out_o, out_q, out_p] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model
 
# generate target given source sequence
def predict_sequence(infenc, infdec, src_o, src_q, src_p, n_steps, cardinality):
	# encode
	state = infenc.predict([src_o, src_q, src_p])
	# start of sequence input
	target_o = np.array([-1]).reshape(1, 1, 1)
	target_q = np.array([-1]).reshape(1, 1, 1)
	target_p = -1
	target_p = to_categorical(target_p, cardinality).reshape((1,1,-1))
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		print(target_o.shape)
		print(target_q.shape)
		print(target_p.shape)
		print(state[0].shape)
		o, q, p, h, c = infdec.predict([target_o, target_q, target_p] + state)
		#print(a)
		# store prediction
		output.append(o[0,0,:])
		output.append(q[0,0,:])
		output.append(p[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_o = o
		target_q = q
		target_p = p
	return np.array(output)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]

In [3]:
#load data
stream_list = []

for directory, subdirectories, files in os.walk('data'):
    for file in files:
        with open('data/' + file, 'r') as f: 
            reader = csv.reader(f)
            sub_list = [list(map(float,rec)) for rec in csv.reader(f, delimiter=',')]
            stream_list = stream_list + sub_list

In [4]:
#create seperate data structures for each variable (offset, quarterlength, pitch)
#normalise offset and quarterlength
offs = []
qlngth = []
ptch = []

offsb = max(element[0] for element in stream_list)
qlngthb = max(element[1] for element in stream_list)
#ptchb = 127.0

for row in stream_list:
    offs.append(row[0]/offsb)
    qlngth.append(row[1]/qlngthb)
    ptch.append(row[2])

In [5]:
#divide the sets in sequences of specific length 
dtlngth=len(stream_list)
seq_length = 6#100 groups of 3

dataX1_o = []
dataX1_q = []
dataX1_p = []
dataX2_o = []
dataX2_q = []
dataX2_p = []

for i in range(0, dtlngth - seq_length + 1, 1):
	seq_in_o = offs[i:i + seq_length]
	seq_in_q = qlngth[i:i + seq_length]
	seq_in_p = ptch[i:i + seq_length]

	# create padded input target sequence
	target_in_o = [-1] + seq_in_o[:-1]
	target_in_q = [-1] + seq_in_q[:-1]
	target_in_p = [-1] + seq_in_p[:-1]
	
	src_encoded = to_categorical(seq_in_p, num_classes=85+1)
	tar2_encoded = to_categorical(target_in_p, num_classes=85+1)
	
	dataX1_o.append([offs for offs in seq_in_o])
	dataX1_q.append([qlngth for qlngth in seq_in_q])
	dataX1_p.append(src_encoded)
	dataX2_o.append([offs for offs in target_in_o])
	dataX2_q.append([qlngth for qlngth in target_in_q])
	dataX2_p.append(tar2_encoded)

n_patterns = len(dataX1_p)
print ("Total Patterns: ", n_patterns)

Total Patterns:  7700


In [6]:
#reshape inputs to be [samples, time steps, features], one hot encode pitch output
dataX1_o = np.reshape(dataX1_o, (n_patterns, seq_length, 1))
dataX1_q = np.reshape(dataX1_q, (n_patterns, seq_length, 1))

dataX2_o = np.reshape(dataX2_o, (n_patterns, seq_length, 1))
dataX2_q = np.reshape(dataX2_q, (n_patterns, seq_length, 1))

In [7]:
# configure problem
n_features = np.array(dataX2_p).shape[2]
n_steps_in = 6
n_steps_out = 6
# define model
train, infenc, infdec = define_models(n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics={'tr_out_o': 'mean_squared_error', 'tr_out_q': 'mean_squared_error', 'tr_out_p': 'accuracy'})

In [8]:
# train model
train.fit([dataX1_o, dataX1_q, np.array(dataX1_p), dataX2_o, dataX2_q, np.array(dataX2_p)], [dataX1_o, dataX1_q, np.array(dataX1_p)], epochs=4)

Epoch 1/4
241/241 [==============================] - 11s 44ms/step - loss: 2.5382 - tr_out_o_loss: 6.3656e-08 - tr_out_q_loss: 5.7358e-09 - tr_out_p_loss: 2.5382 - tr_out_o_mean_squared_error: 0.0989 - tr_out_q_mean_squared_error: 0.2541 - tr_out_p_accuracy: 0.2518
Epoch 2/4
241/241 [==============================] - 11s 44ms/step - loss: 1.2850 - tr_out_o_loss: 6.3656e-08 - tr_out_q_loss: 5.7358e-09 - tr_out_p_loss: 1.2850 - tr_out_o_mean_squared_error: 0.0911 - tr_out_q_mean_squared_error: 0.2626 - tr_out_p_accuracy: 0.5947
Epoch 3/4
241/241 [==============================] - 10s 42ms/step - loss: 0.6033 - tr_out_o_loss: 6.3656e-08 - tr_out_q_loss: 5.7358e-09 - tr_out_p_loss: 0.6033 - tr_out_o_mean_squared_error: 0.0954 - tr_out_q_mean_squared_error: 0.2592 - tr_out_p_accuracy: 0.8536
Epoch 4/4
241/241 [==============================] - 11s 45ms/step - loss: 0.3091 - tr_out_o_loss: 6.3656e-08 - tr_out_q_loss: 5.7358e-09 - tr_out_p_loss: 0.3091 - tr_out_o_mean_squared_error: 0.0985 - 

In [ ]:
infdec.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None, 86)]   0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, None, 88)     0           input_4[0][0]                    
                                                                 input_5[0][0]         

In [12]:
# spot check some examples
#for _ in range(10):
i = randint(1, 1000)
X1_o = np.reshape(dataX1_o[i], (1, seq_length, 1))
X1_q = np.reshape(dataX1_q[i], (1, seq_length, 1))
X1_p = np.reshape(dataX1_p[i], (1, seq_length, 86))
target = predict_sequence(infenc, infdec, X1_o, X1_q, X1_p, n_steps_out, n_features)
for j in range(seq_length):
    print('X_o=%s, X_q=%s, X_p=%s, y_o=%s, y_q=%s, y_p=%s' % (dataX1_o[i][j], dataX1_q[i][j], one_hot_decode([dataX1_p[i][j]]), target[3*j], target[3*j+1], one_hot_decode([target[3*j+2]])))

(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
(1, 1, 1)
(1, 1, 1)
(1, 1, 86)
(1, 128)
X_o=[0.12606838], X_q=[0.07317073], X_p=[67], y_o=[0.55938846], y_q=[0.37878808], y_p=[8]
X_o=[0.12606838], X_q=[0.02439024], X_p=[55], y_o=[0.54453313], y_q=[0.33581254], y_p=[79]
X_o=[0.12606838], X_q=[0.02439024], X_p=[55], y_o=[0.51103514], y_q=[0.3201807], y_p=[75]
X_o=[0.12713675], X_q=[0.06504065], X_p=[62], y_o=[0.45501176], y_q=[0.31633216], y_p=[83]
X_o=[0.12713675], X_q=[0.04878049], X_p=[31], y_o=[0.38875005], y_q=[0.33068424], y_p=[83]
X_o=[0.12713675], X_q=[0.02439024], X_p=[59], y_o=[0.35591644], y_q=[0.36264336], y_p=[11]


In [ ]:
target[3]

array([0.20503959], dtype=float32)

In [ ]:
X1_o = np.reshape(dataX1_o[2], (1, 2, 1))
X1_q = np.reshape(dataX1_q[2], (1, 2, 1))
X1_p = np.reshape(dataX1_p[2], (1, 2, 86))
state = infenc.predict([X1_o, X1_q, X1_p])
state
#target_o = np.array([-1]).reshape(1, 1)
#target_q = np.array([-1]).reshape(1, 1)
#target_p = -1.0
#target_p = np_utils.to_categorical(target_p, num_classes=86)
#target_p = np.array(target_p.reshape(1, 86)
#yhat, h, c = infdec.predict([target_o, target_q, target_p] + state)